In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from ipywidgets import interact, Dropdown
from IPython.display import display


In [15]:
df_co2 = pd.read_csv('data/co2.csv')
df_co2['year'] = df_co2['year'].astype(int)
country_options = sorted(df_co2['country_code'].unique())
print(f'Loaded {len(df_co2)} rows for {len(country_options)} countries.')


Loaded 9466 rows for 206 countries.


In [19]:
def plot_hw_forecast(country_code):
    country_data = df_co2[df_co2['country_code'] == country_code].sort_values('year')
    if country_data.empty:
        raise ValueError(f'No data found for {country_code}')
    train = country_data[(country_data['year'] >= 1970) & (country_data['year'] <= 2019)]
    if len(train) < 15:
        raise ValueError(f"Not enough training data for {country_code}")
    ts = train['co2'].values
    model = ExponentialSmoothing(ts, trend='add', seasonal=None)
    fitted = model.fit(optimized=True)
    forecast_steps = 6
    forecast = fitted.forecast(steps=forecast_steps)
    forecast_values = forecast.values if hasattr(forecast, 'values') else np.asarray(forecast)
    forecast_years = np.arange(2019, 2019 + forecast_steps)
    if len(forecast_values) > 0:
        if last_year := train['year'].iloc[-1]:
            forecast_values = forecast_values.copy()
            forecast_values[0] = train['co2'].iloc[-1]
    display_years = np.arange(train['year'].iloc[0], 2020)
    actual_series = train.set_index('year')['co2'].reindex(display_years, method='ffill')
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(actual_series.index, actual_series.values, label='Actual (1970-2019)', color='tab:blue', linewidth=2)
    last_year = actual_series.index[-1]
    last_value = actual_series.iloc[-1]
    ax.scatter([last_year], [last_value], color='tab:blue', s=100, edgecolor='white', label=f'Last actual ({last_year})')
    ax.plot(forecast_years, forecast_values, label='Forecast (2019-2024)', color='tab:red', marker='o', linestyle='--')
    ax.set_title(f'Holt-Winters CO2 Forecast for {country_code}')
    ax.set_xlabel('Year')
    ax.set_ylabel('CO2 Emissions')
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    plt.show()
    forecast_df = pd.DataFrame({'year': forecast_years, 'holt_winters_forecast': forecast.round(2)})
    display(forecast_df)


In [20]:
interact(plot_hw_forecast, country_code=Dropdown(options=country_options, value=country_options[0], description='Country'))

interactive(children=(Dropdown(description='Country', options=('ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG'…

<function __main__.plot_hw_forecast(country_code)>